<pre>
실습문제

소방청 > 기관소개 > 전국소방기관 안내 에서
검색 박스에 '서울'을 입력하고 검색 버튼을 눌러서
서울시내 소재하는 100군데의 소방서, 안전센터를 찾아서
서울시 지도에 표시하세요.

1. (크롤링) 셀레니움으로 크롤링을 해서 소방서명, 주소, 전화번호  
   데이터를 가져와서 데이터프레임을 만든 후,
   데이터프레임에 새로운 컬럼 안전센터여부를 만들고 소방서는 0,
   안전센터는 1로 채우세요. (40점)

2. (OpenAPI) 앞에서 구한 주소를 가지고, 카카오 로컬 Open API를
   이용하여 소방서의 위도, 경도 정보를 구한 후 데이터프레임에
   추가하세요. (30점)

3. (지도시각화) 앞에서 구한 정보를 바탕으로 서울 지도(Open Street Map)에
   표시하되, 다음의 조건을 만족시키세요. (30점)
	- 툴팁에는 소방서명과 전화번호가 표시되어야 함
	- 팝업에는 소방서 주소가 가로로 표시되어야 함
	- 소방서와 안전센터의 아이콘이 다르게 표시되어야 함
	- 서울소재 소방서 위치라는 제목이 표시되어야 함
   
</pre>

In [3]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup

In [37]:
base_url = 'https://www.nfa.go.kr/nfa/introduce/status/firestationidfo/'
sub_url = '?searchDistance=10&searchMode=keyword&myX=37.5390208&myY=127.0513664&searchKeyword=서울'
driver = webdriver.Chrome('05.Crawling/chromedriver.exe')
driver.get(base_url + sub_url)
time.sleep(2)

In [17]:
soup = BeautifulSoup(driver.page_source, 'html.parser')
lis = soup.select('.stations-list > li')
len(lis)

10

In [29]:
# 소방서명
li = trs[1]
li.select_one('.title').get_text().strip()

'서대문소방서'

In [30]:
# 주소
li.select_one('li > address').get_text().strip()

'서울특별시 서대문구 연희로 182(연희동)'

In [31]:
# 전화번호
li.select_one('.tel').get_text().strip()

'02-3144-1190'

In [26]:
title = li.select_one('.title').get_text().strip()
addr = li.select_one('li > address').get_text().strip()
tel = li.select_one('.tel').get_text().strip()
title , addr , tel

('동작소방서', '서울특별시 동작구 여의대방로16길 55(신대방동)', '02-847-1190')

In [34]:
lines = [] 
for li in lis:
    title = li.select_one('.title').get_text().strip()
    addr = li.select_one('li > address').get_text().strip()
    tel = li.select_one('.tel').get_text().strip()
    lines.append({'소방서명':title,'주소':addr,'전화번호':tel})
lines

[{'소방서명': '동작소방서', '주소': '서울특별시 동작구 여의대방로16길 55(신대방동)', '전화번호': '02-847-1190'},
 {'소방서명': '서대문소방서', '주소': '서울특별시 서대문구 연희로 182(연희동)', '전화번호': '02-3144-1190'},
 {'소방서명': '광진소방서', '주소': '서울특별시 광진구 광나루로 480(구의동)', '전화번호': '02-457-0119'},
 {'소방서명': '송파소방서', '주소': '서울특별시 송파구 오금로51길 56(마천동)', '전화번호': '02-403-2119'},
 {'소방서명': '양천소방서', '주소': '서울특별시 양천구 목동서로 180(목동)', '전화번호': '02-2655-1119'},
 {'소방서명': '은평소방서', '주소': '서울특별시 은평구 통일로 962(진관동)', '전화번호': '02-355-0119'},
 {'소방서명': '노원소방서', '주소': '서울특별시 노원구 한글비석로 1길 8(하계동)', '전화번호': '02-977-0119'},
 {'소방서명': '종로소방서', '주소': '서울특별시 종로구 종로1길 28(수송동)', '전화번호': '02-735-6119'},
 {'소방서명': '도봉소방서', '주소': '서울특별시 도봉구 도봉로 666(방학동)', '전화번호': '02-3492-3437'},
 {'소방서명': '용산소방서', '주소': '서울특별시 용산구 한강대로 167(한강로2가)', '전화번호': '02-794-0119'}]

In [40]:
lines = [] 
for page in range(1,11):
    base_url = 'https://www.nfa.go.kr/nfa/introduce/status/firestationidfo/'
    sub_url = '?searchDistance=10&searchMode=keyword&myX=37.5390208&myY=127.0513664&searchKeyword=서울'
    driver = webdriver.Chrome('05.Crawling/chromedriver.exe')
    url = f"{base_url}{sub_url}&pageIndex={page}"
    driver.get(url)
    time.sleep(2)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    lis = soup.select('.stations-list > li')
    for li in lis:
        title = li.select_one('.title').get_text().strip()
        addr = li.select_one('li > address').get_text().strip()
        tel = li.select_one('.tel').get_text().strip()
        lines.append({'소방서명':title,'주소':addr,'전화번호':tel})
        time.sleep(1)
    driver.close()


In [41]:
df = pd.DataFrame(lines)

In [43]:
df.tail()

,소방서명,주소,전화번호
95,송파소방서-운동장-119 안전센터,서울특별시 송파구 올림픽로 25 (잠실동),02-2203-2380
96,송파소방서-잠실-119 안전센터,서울특별시 송파구 석촌호수로 151 (잠실동),02-422-0119
97,송파소방서-방이-119 안전센터,서울특별시 송파구 강동대로 286 (방이동),02-409-0059
98,송파소방서-거여-119 안전센터,서울특별시 송파구 마천로 329 (마천동),02-400-0119
99,송파소방서-마천-119 안전센터,서울특별시 송파구 오금로51길 56 (마천동),02-3401-2119


In [92]:
safe = df['소방서명'].str.split()
df['안전센터여부'] = safe.str.get(-1)


In [93]:
df.head(3) ,df.tail(3)

(     소방서명                           주소          전화번호  안전센터여부
 0   동작소방서  서울특별시 동작구 여의대방로16길 55(신대방동)   02-847-1190   동작소방서
 1  서대문소방서      서울특별시 서대문구 연희로 182(연희동)  02-3144-1190  서대문소방서
 2   광진소방서      서울특별시 광진구 광나루로 480(구의동)   02-457-0119   광진소방서,
                  소방서명                         주소          전화번호 안전센터여부
 97  송파소방서-방이-119 안전센터   서울특별시 송파구 강동대로 286 (방이동)   02-409-0059   안전센터
 98  송파소방서-거여-119 안전센터    서울특별시 송파구 마천로 329 (마천동)   02-400-0119   안전센터
 99  송파소방서-마천-119 안전센터  서울특별시 송파구 오금로51길 56 (마천동)  02-3401-2119   안전센터)

In [96]:
df['안전센터여부'] = np.where(df['안전센터여부'] == '안전센터', '1' ,'0')

In [98]:
df.head()

,소방서명,주소,전화번호,안전센터여부
0,동작소방서,서울특별시 동작구 여의대방로16길 55(신대방동),02-847-1190,0
1,서대문소방서,서울특별시 서대문구 연희로 182(연희동),02-3144-1190,0
2,광진소방서,서울특별시 광진구 광나루로 480(구의동),02-457-0119,0
3,송파소방서,서울특별시 송파구 오금로51길 56(마천동),02-403-2119,0
4,양천소방서,서울특별시 양천구 목동서로 180(목동),02-2655-1119,0


In [99]:
df.tail()

,소방서명,주소,전화번호,안전센터여부
95,송파소방서-운동장-119 안전센터,서울특별시 송파구 올림픽로 25 (잠실동),02-2203-2380,1
96,송파소방서-잠실-119 안전센터,서울특별시 송파구 석촌호수로 151 (잠실동),02-422-0119,1
97,송파소방서-방이-119 안전센터,서울특별시 송파구 강동대로 286 (방이동),02-409-0059,1
98,송파소방서-거여-119 안전센터,서울특별시 송파구 마천로 329 (마천동),02-400-0119,1
99,송파소방서-마천-119 안전센터,서울특별시 송파구 오금로51길 56 (마천동),02-3401-2119,1


In [169]:
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rcParams['axes.unicode_minus'] = False
mpl.rcParams['font.family'] = 'Malgun Gothic'
from urllib.parse import quote
import json
import requests
import folium

In [118]:
with open('04.지도시각화/kakaoapikey.txt') as file:
    kakao_key = file.read()
len(kakao_key)

32

In [160]:
local_url = 'https://dapi.kakao.com/v2/local/search/address.json'
addr = '서울특별시 동작구 여의대방로16길 55(신대방동)'
url = f'{local_url}?query={quote(addr)}'
url

'https://dapi.kakao.com/v2/local/search/address.json?query=%EC%84%9C%EC%9A%B8%ED%8A%B9%EB%B3%84%EC%8B%9C%20%EB%8F%99%EC%9E%91%EA%B5%AC%20%EC%97%AC%EC%9D%98%EB%8C%80%EB%B0%A9%EB%A1%9C16%EA%B8%B8%2055%28%EC%8B%A0%EB%8C%80%EB%B0%A9%EB%8F%99%29'

In [161]:
header = {'Authorization': f'KakaoAK {kakao_key}'}

In [162]:

result = requests.get(url, headers=header).json()
result

{'documents': [{'address': {'address_name': '서울 동작구 신대방동 460-19',
    'b_code': '1159010900',
    'h_code': '1159068000',
    'main_address_no': '460',
    'mountain_yn': 'N',
    'region_1depth_name': '서울',
    'region_2depth_name': '동작구',
    'region_3depth_h_name': '신대방2동',
    'region_3depth_name': '신대방동',
    'sub_address_no': '19',
    'x': '126.91771856513',
    'y': '37.4946721349965'},
   'address_name': '서울 동작구 여의대방로16길 55',
   'address_type': 'ROAD_ADDR',
   'road_address': {'address_name': '서울 동작구 여의대방로16길 55',
    'building_name': '동작소방서',
    'main_building_no': '55',
    'region_1depth_name': '서울',
    'region_2depth_name': '동작구',
    'region_3depth_name': '신대방동',
    'road_name': '여의대방로16길',
    'sub_building_no': '',
    'underground_yn': 'N',
    'x': '126.91771856513',
    'y': '37.4946721349965',
    'zone_no': '07062'},
   'x': '126.91771856513',
   'y': '37.4946721349965'}],
 'meta': {'is_end': True, 'pageable_count': 1, 'total_count': 1}}

In [163]:
lng = float(result['documents'][0]['x'])
lat = float(result['documents'][0]['y'])
lng, lat

(126.91771856513, 37.4946721349965)

In [164]:
df.to_csv('소방청.csv', index=False)

In [144]:
df.index

RangeIndex(start=0, stop=100, step=1)

In [205]:
# 은평소방서-수색-119 안전센터 의 주소를 변경  ->  서울특별시 은평구 수색로 320
df = pd.read_csv('소방청.csv')

In [206]:
lng_list, lat_list = [],[]
for i in df.index:
    url = f'{local_url}?query={quote(df.주소[i])}'
    result = requests.get(url, headers=header).json()
    lng_list.append(float(result['documents'][0]['x']))
    lat_list.append(float(result['documents'][0]['y']))



In [207]:
df['위도']= lat_list
df['경도']= lng_list
df.tail()

,소방서명,주소,전화번호,안전센터여부,위도,경도
95,송파소방서-운동장-119 안전센터,서울특별시 송파구 올림픽로 25 (잠실동),02-2203-2380,1,37.516200,127.075940
96,송파소방서-잠실-119 안전센터,서울특별시 송파구 석촌호수로 151 (잠실동),02-422-0119,1,37.507316,127.093893
97,송파소방서-방이-119 안전센터,서울특별시 송파구 강동대로 286 (방이동),02-409-0059,1,37.519785,127.136572
98,송파소방서-거여-119 안전센터,서울특별시 송파구 마천로 329 (마천동),02-400-0119,1,37.494163,127.151828
99,송파소방서-마천-119 안전센터,서울특별시 송파구 오금로51길 56 (마천동),02-3401-2119,1,37.499833,127.142592


In [208]:
df.head()

,소방서명,주소,전화번호,안전센터여부,위도,경도
0,동작소방서,서울특별시 동작구 여의대방로16길 55(신대방동),02-847-1190,0,37.494672,126.917719
1,서대문소방서,서울특별시 서대문구 연희로 182(연희동),02-3144-1190,0,37.573205,126.935996
2,광진소방서,서울특별시 광진구 광나루로 480(구의동),02-457-0119,0,37.544826,127.082779
3,송파소방서,서울특별시 송파구 오금로51길 56(마천동),02-403-2119,0,37.499833,127.142592
4,양천소방서,서울특별시 양천구 목동서로 180(목동),02-2655-1119,0,37.530236,126.872342


In [225]:
map = folium.Map(location=[df.위도.mean(),df.경도.mean()], zoom_start=12)
for i in df.index:
    if df.안전센터여부[i] == 1:
        folium.Marker(
            location=[df.위도[i],df.경도[i]],
            popup=folium.Popup(df.주소[i], max_width=300),
            tooltip=(df.소방서명[i], df.전화번호[i]),
            icon=folium.Icon(color='pink',icon='glyphicon glyphicon-earphone',icon_size=17)
        ).add_to(map)
    else: 
        folium.Marker(
            location=[df.위도[i],df.경도[i]],
            popup=folium.Popup(df.주소[i], max_width=300),
            tooltip=(df.소방서명[i], df.전화번호[i]),
            icon=folium.Icon(color='red',icon='glyphicon glyphicon-fire')
        ).add_to(map)
title_html = '<h3 align="center" style="font-size:20px">서울소재 소방서 위치</h3>' 
map.get_root().html.add_child(folium.Element(title_html))
map